# Tutorial 10: Real-world Applications

In this tutorial, we'll apply the concepts learned in previous sessions to build practical, real-world applications using LangChain and LangGraph. We'll implement four different applications to showcase the versatility and power of these libraries in solving complex problems.

## Setup

First, let's import the necessary libraries and set up our environment:

In [1]:
import os
from typing import Dict, List, Any
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, END

# Set up the Groq LLM
# Initialize the Groq LLM
llm = ChatGroq(
        model_name="llama-3.1-70b-versatile",
        temperature=0.1,
        model_kwargs={"top_p": 0.5, "seed": 1337}
    )
# Define our state structure
class State(dict):
    messages: List[Dict[str, str]]
    memory: Any
    current_step: str

## 1. Building a content moderation system

Let's create a content moderation system that can classify text as safe or potentially harmful.

In [3]:
moderation_prompt = ChatPromptTemplate.from_template(
    """You are a content moderation AI. Your task is to classify the following text as either 'SAFE' or 'UNSAFE'. 
    If the content contains any hate speech, explicit violence, or adult content, classify it as 'UNSAFE'. 
    Otherwise, classify it as 'SAFE'. Respond with only 'SAFE' or 'UNSAFE'.

    Text to moderate: {text}
    """
)

moderation_chain = LLMChain(llm=llm, prompt=moderation_prompt)

def moderate_content(state: State) -> State:
    text = state["messages"][-1]["content"]
    result = moderation_chain.run(text=text)
    state["messages"].append({"role": "assistant", "content": f"Content classification: {result}"})
    return state

# Create the graph
moderation_graph = StateGraph(State)
moderation_graph.add_node("moderate", moderate_content)
moderation_graph.set_entry_point("moderate")
moderation_graph.add_edge("moderate", END)

# Compile the graph
moderation_app = moderation_graph.compile()

# Test the moderation system
initial_state = State(messages=[], memory=None, current_step="")
initial_state["messages"].append({"role": "human", "content": "Hello, how are you today?"})
for event in moderation_app.stream(initial_state):
    for output in event.values():
        print(output["messages"][-1]["content"])

initial_state["messages"] = [{"role": "human", "content": "I hate everyone and wish them harm!"}]
for event in moderation_app.stream(initial_state):
    for output in event.values():
        print(output["messages"][-1]["content"])

Content classification: SAFE
Content classification: UNSAFE


## 2. Implementing a language translation service

Now, let's create a language translation service that can translate text between multiple languages.

In [11]:
translation_prompt = ChatPromptTemplate.from_template(
    """Translate the following text from {source_lang} to {target_lang}:
    
    Text: {text}
    
    Translation:
    """
)

translation_chain = LLMChain(llm=llm, prompt=translation_prompt)

def translate_text(state: State) -> State:
    text = state["messages"][-1]["content"]
    source_lang = state["source_lang"]
    target_lang = state["target_lang"]
    result = translation_chain.run(text=text, source_lang=source_lang, target_lang=target_lang)
    state["messages"].append({"role": "assistant", "content": f"Translation: {result}"})
    return state

# Create the graph
translation_graph = StateGraph(State)
translation_graph.add_node("translate", translate_text)
translation_graph.set_entry_point("translate")
translation_graph.add_edge("translate", END)

# Compile the graph
translation_app = translation_graph.compile()

# Test the translation service
initial_state = State(messages=[], memory=None, current_step="", source_lang="English", target_lang="French")
initial_state["messages"].append({"role": "human", "content": "Hello, how are you?"})
for event in translation_app.stream(initial_state):
    for output in event.values():
        print(output["messages"][-1]["content"])

initial_state = State(messages=[], memory=None, current_step="", source_lang="English", target_lang="Spanish")
initial_state["messages"].append({"role": "human", "content": "I love learning new languages!"})
for event in translation_app.stream(initial_state):
    for output in event.values():
        print(output["messages"][-1]["content"])

Translation: Bonjour, comment allez-vous ?
Translation: Me encanta aprender nuevos idiomas.


## 3. Creating an automated customer support chatbot

Let's build a customer support chatbot that can handle common queries and provide relevant information.

In [13]:
support_prompt = ChatPromptTemplate.from_template(
    """You are a customer support AI for a software company. Answer the following query based on these guidelines:
    1. Be polite and professional
    2. If you don't know the answer, say so and offer to connect the customer with a human representative
    3. For technical issues, provide step-by-step troubleshooting instructions
    4. For pricing or account questions, provide general information and links to relevant pages

    Customer query: {query}

    Your response:
    """
)

support_chain = LLMChain(llm=llm, prompt=support_prompt)

def handle_query(state: State) -> State:
    query = state["messages"][-1]["content"]
    result = support_chain.run(query=query)
    state["messages"].append({"role": "assistant", "content": result})
    return state

# Create the graph
support_graph = StateGraph(State)
support_graph.add_node("handle_query", handle_query)
support_graph.set_entry_point("handle_query")
support_graph.add_edge("handle_query", END)

# Compile the graph
support_app = support_graph.compile()

# Test the customer support chatbot
initial_state = State(messages=[], memory=None, current_step="")
initial_state["messages"].append({"role": "human", "content": "How do I reset my password?"})
for event in support_app.stream(initial_state):
    for output in event.values():    
        print(output["messages"][-1]["content"])

initial_state["messages"] = [{"role": "human", "content": "What are your pricing plans?"}]
for output in support_app.stream(initial_state):
    for output in event.values():
        print(output["messages"][-1]["content"])

Dear valued customer,

Thank you for reaching out to us for assistance. I'd be happy to help you with resetting your password. Here are the step-by-step instructions to follow:

1. Go to our login page and click on the "Forgot Password" link.
2. Enter your registered email address and click on the "Send Reset Link" button.
3. Check your email inbox for a password reset email from our company. If you don't see the email, please check your spam folder as well.
4. Click on the password reset link provided in the email. This will take you to a new page where you can enter your new password.
5. Enter your new password and confirm it by re-entering it in the next field.
6. Click on the "Reset Password" button to complete the process.

If you encounter any issues during the password reset process, please try the following troubleshooting steps:

- Ensure that you are using the correct email address associated with your account.
- Check that your email account is not blocking emails from our c

## Conclusion

In this tutorial, we've built four real-world applications using LangChain and LangGraph:

1. A content moderation system
2. A language translation service
3. An automated customer support chatbot
4. A text-based adventure game with AI-driven narrative

These applications demonstrate the versatility and power of combining LangChain components with LangGraph flows. Here are some key takeaways:

- LangChain's `LLMChain` allows for easy creation of task-specific AI assistants.
- LangGraph's `StateGraph` enables the creation of complex, stateful workflows.
- The combination of LangChain and LangGraph allows for the development of sophisticated AI applications that can handle a wide range of tasks.

## Potential Improvements and Extensions

1. Content Moderation System:
   - Implement more fine-grained content categories (e.g., violence, hate speech, adult content)
   - Add a confidence score to the classification
   - Integrate with a database to log moderation decisions

2. Language Translation Service:
   - Add language detection to automatically identify the source language
   - Implement translation memory to improve consistency across similar phrases
   - Add support for idiomatic expressions and cultural context

3. Customer Support Chatbot:
   - Integrate with a knowledge base for more accurate responses
   - Implement sentiment analysis to detect customer frustration
   - Add a handoff mechanism to transfer complex queries to human agents

4. Text-based Adventure Game:
   - Implement a more complex state management system to track player inventory, health, etc.
   - Add multiple endings based on player choices
   - Integrate with a database to save and load game progress

## Next Steps

As you continue to work with LangChain and LangGraph, consider the following:

- Experiment with different LLM providers to compare performance and capabilities
- Explore more advanced LangChain components like agents and tools
- Implement more complex LangGraph flows with parallel processing and advanced conditional logic
- Consider deploying these applications as web services using frameworks like FastAPI or Flask

Remember to always consider ethical implications and potential biases when developing AI applications, and implement appropriate safeguards and monitoring systems.